In [1]:
import pandas as pd
import numpy as np
import os
import sys
import datetime 
from functools import partial
import math

In [2]:
folder = os.getcwd() + '/datasets/'
files = os.listdir(folder)

for file_name in files.copy():
    if '.txt' in file_name:
        files.remove(file_name)

In [3]:
dfs : dict[pd.DataFrame] = {}

for file_name in files:
    dfs[file_name] = pd.read_csv(folder+file_name)

In [4]:
def convert_dates(text:str):
    return datetime.datetime.strptime(text,'%Y-%m-%dT%H:%M:%S.%fZ')

In [5]:
years = []
for key,value in dfs.items():
    date_col_name = value.columns[0]
    try:
        value[date_col_name] = value[date_col_name].map(convert_dates)
    except TypeError:
        pass
    finally:
        years.append(min(value[date_col_name].dt.year))

In [6]:
max(years)

2024

In [7]:
for key,value in dfs.items():
    date_col_name = value.columns[0]
    try:
        value[date_col_name] = value[date_col_name].map(convert_dates)
    except TypeError:
        pass
    finally:
        if min(value[date_col_name].dt.year) == 2024:
            print(key)

Sentinel-5P HCHO-HCHO_VISUALIZED-2024-07-01T00_00_00.000Z-2024-10-01T23_59_59.999Z.csv


In [8]:
for key,value in dfs.copy().items():
    date_col_name = value.columns[0]
    try:
        value[date_col_name] = value[date_col_name].map(convert_dates)
    except TypeError:
        pass
    finally:
        if 2024 not in value[date_col_name].dt.year.to_list():
            del dfs[key]

In [9]:
for key,value in dfs.copy().items():
    date_col_name = value.columns[0]
    dfs[key] = value[value[date_col_name].dt.year == 2024]

In [10]:
_ = [print(len(value)) for value in dfs.values()]

32
274
49
213
32
46
46
224
264
273
273
213
65
46
46
49
215
88
38
32
213
46
46
270


In [11]:
intersected_dates = []

In [12]:
for values in dfs.values():
    date_col_name = values.columns[0]
    intersected_dates.extend(values[date_col_name].map(str).to_list())

In [13]:
intersected_dates = set(intersected_dates)

In [14]:
len(intersected_dates)

276

In [15]:
def delete_not_intersected_day(day):
    if str(day) not in intersected_dates:
        return np.nan
    return day

In [16]:
for key,values in dfs.copy().items():
    date_col_name = values.columns[0]
    values[date_col_name] = values[date_col_name].map(delete_not_intersected_day)
    dfs[key] = values[values[date_col_name] != np.nan]

/tmp/ipykernel_96108/1537659331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values[date_col_name] = values[date_col_name].map(delete_not_intersected_day)


In [17]:
for values in dfs.values():
    _,*col_names = list(values.columns)
    col_names.insert(0,'day')
    values.columns = col_names

In [18]:
for key,values in dfs.copy().items():
    dfs[key]['day'] = values['day'].map(str)
    dfs[key]['day'] = values['day'].map(lambda x : x.split(' ')[0])
    dfs[key]['day'] = values['day'].map(lambda x : ''.join(x.split('-')[:-1]))

In [19]:
dfs_means = []
for key,values in dfs.copy().items():
    unique_months = values['day'].unique()
    
    df_unique_months = {}
    for month in unique_months:
        unique_month_values = values[values['day'] == month]
        unique_month_values = unique_month_values.drop('day',axis=1)
        
        for mean_value,index in zip(unique_month_values.mean(),unique_month_values.mean().index):
            if df_unique_months.get(index,False):
                df_unique_months[month].append(mean_value)
            else:
                df_unique_months[month] = [mean_value]
    df_unique_months = pd.DataFrame(data=df_unique_months,index=[0])
    dfs_means.append(df_unique_months)

In [22]:
len(dfs_means)

2

In [21]:
dfs_means['202409'].to_csv('./updated_datasets_distrubutions/202409.csv',index=False)
dfs_means['202410'].to_csv('./updated_datasets_distrubutions/202410.csv',index=False)